In [1]:
from typing import Tuple
import sys
sys.path.append("..")

import numpy as np
import torch
from alive_progress import alive_it
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src.hooks import HookManager
from src.utils import list_child_modules


class SampleDataset(Dataset):
    def __init__(self, features: np.ndarray, label: np.ndarray) -> None:
        super().__init__()
        self.features = features
        self.label = label
        self._scaler()

    def _scaler(self) -> None:
        mm = MinMaxScaler()
        self.features = mm.fit_transform(self.features)

    def __len__(self) -> int:
        return self.features.shape[0]

    def __getitem__(self, idx: int) -> Tuple[np.ndarray, np.ndarray]:
        return (self.features[idx, :], self.label[idx] - 1)


class FFN(nn.Module):
    def __init__(self, n_feats: int, n_classes: int) -> None:
        super().__init__()
        self.ffn = nn.Sequential(
            nn.LayerNorm(n_feats),
            nn.Dropout(0.1),
            nn.Linear(n_feats, 512),
            nn.ReLU(),
            nn.LayerNorm(512),
            nn.Dropout(0.1),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.LayerNorm(512),
            nn.Dropout(0.1),
            nn.Linear(512, n_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.ffn(x)

In [3]:
cov_type = fetch_covtype(data_home="../covertype")
X_train, X_test, y_train, y_test = train_test_split(
    cov_type.data, cov_type.target, test_size=0.2, random_state=42
)
train_dataset = SampleDataset(X_train, y_train)
test_dataset = SampleDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print("training...")

model = FFN(X_train.shape[1], len(set(y_train)))
list_child_modules(model)

n_epochs = 1
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
model = FFN(X_train.shape[1], len(set(y_train)))

for epoch in alive_it(range(n_epochs), bar="fish"):
    epoch_loss = 0
    model.train()
    for tr_x, tr_y in train_loader:
        tr_x, tr_y = (
            torch.tensor(tr_x, dtype=torch.float32),
            torch.tensor(tr_y, dtype=torch.long),
        )
        pred_tr_y = model(tr_x)
        loss = criterion(pred_tr_y, tr_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch + 1} | Loss: {epoch_loss / len(train_loader)}")

print("testing...")
model.eval()
hook_mgr = HookManager()
hook_mgr.register_hooks(model, partial_matches=None)

with torch.no_grad():
    preds = []

    for ts_x, ts_y in test_loader:
        ts_x = torch.tensor(ts_x, dtype=torch.float32)
        ts_y = torch.tensor(ts_y, dtype=torch.long)
        pred_y_test = model(ts_x)
        preds.append(pred_y_test)

    print(
        f"Accuracy: {np.sum(np.argmax(np.array(preds)[:, 0, :], axis=1) == y_test).item() / len(y_test)}"
    )

activations = hook_mgr.get_activations()
# print(activations)


training...
ffn.0 LayerNorm((54,), eps=1e-05, elementwise_affine=True)
ffn.1 Dropout(p=0.1, inplace=False)
ffn.2 Linear(in_features=54, out_features=512, bias=True)
ffn.3 ReLU()
ffn.4 LayerNorm((512,), eps=1e-05, elementwise_affine=True)
ffn.5 Dropout(p=0.1, inplace=False)
ffn.6 Linear(in_features=512, out_features=512, bias=True)
ffn.7 ReLU()
ffn.8 LayerNorm((512,), eps=1e-05, elementwise_affine=True)
ffn.9 Dropout(p=0.1, inplace=False)
ffn.10 Linear(in_features=512, out_features=7, bias=True)
ffn.11 Softmax(dim=1)
on 0: Epoch: 1 | Loss: 1.958906238029503


on 0: /var/folders/5p/5d5cjs1n0sd8dhmpcjd35h0r0000gn/T/ipykernel_34191/456188075.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
        torch.tensor(tr_x, dtype=torch.float32),
      /var/folders/5p/5d5cjs1n0sd8dhmpcjd35h0r0000gn/T/ipykernel_34191/456188075.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
        torch.tensor(tr_y, dtype=torch.long),


|¸.·´¯`·.·´¯`·.¸¸.·´¯`·.¸.·´¯`·.·´¯`·.¸¸.| 1/1 [100%] in 17.5s (0.06/s) 
testing...


/var/folders/5p/5d5cjs1n0sd8dhmpcjd35h0r0000gn/T/ipykernel_34191/456188075.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ts_x = torch.tensor(ts_x, dtype=torch.float32)
/var/folders/5p/5d5cjs1n0sd8dhmpcjd35h0r0000gn/T/ipykernel_34191/456188075.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ts_y = torch.tensor(ts_y, dtype=torch.long)


Accuracy: 0.043105599683312824


In [4]:
from src.statistics import print_summary

for k, v in activations.items():
    print_summary(k, v)


 Stats for ffn.0:
  mean: -0.0000
  std: 0.9999
  min: -0.4870
  max: 2.8785
  sparsity: 0.0000
  skewness: 1.8789
  kurtosis: 2.0993

 Stats for ffn.1:
  mean: -0.0000
  std: 0.9999
  min: -0.4870
  max: 2.8785
  sparsity: 0.0000
  skewness: 1.8789
  kurtosis: 2.0993

 Stats for ffn.2:
  mean: -0.0055
  std: 0.5766
  min: -1.6163
  max: 1.7567
  sparsity: 0.0000
  skewness: 0.1820
  kurtosis: -0.0047

 Stats for ffn.3:
  mean: 0.2253
  std: 0.3476
  min: 0.0000
  max: 1.7567
  sparsity: 0.5000
  skewness: 1.7828
  kurtosis: 2.8011

 Stats for ffn.4:
  mean: -0.0000
  std: 1.0000
  min: -0.6482
  max: 4.4047
  sparsity: 0.0000
  skewness: 1.7828
  kurtosis: 2.8011

 Stats for ffn.5:
  mean: -0.0000
  std: 1.0000
  min: -0.6482
  max: 4.4047
  sparsity: 0.0000
  skewness: 1.7828
  kurtosis: 2.8011

 Stats for ffn.6:
  mean: -0.0097
  std: 0.5436
  min: -1.5103
  max: 1.5893
  sparsity: 0.0000
  skewness: -0.0666
  kurtosis: -0.0989

 Stats for ffn.7:
  mean: 0.2124
  std: 0.3091
  min:

In [6]:
list_child_modules(model)

ffn.0 LayerNorm((54,), eps=1e-05, elementwise_affine=True)
ffn.1 Dropout(p=0.1, inplace=False)
ffn.2 Linear(in_features=54, out_features=512, bias=True)
ffn.3 ReLU()
ffn.4 LayerNorm((512,), eps=1e-05, elementwise_affine=True)
ffn.5 Dropout(p=0.1, inplace=False)
ffn.6 Linear(in_features=512, out_features=512, bias=True)
ffn.7 ReLU()
ffn.8 LayerNorm((512,), eps=1e-05, elementwise_affine=True)
ffn.9 Dropout(p=0.1, inplace=False)
ffn.10 Linear(in_features=512, out_features=7, bias=True)
ffn.11 Softmax(dim=1)
